# Imports and functions

In [1]:
import pandas as pd
import py_entitymatching as em
import py_stringmatching as sm

from math import radians, sin, cos, asin, sqrt

# Read CSV Files

In [ ]:
# Other terms: restaurant, attraction, hotel
term = 'restaurant'

In [2]:
A = em.read_csv_metadata('./../data/yelp_{}.csv'.format(term), key='id', low_memory=False)
B = em.read_csv_metadata('./../data/google_{}.csv'.format(term), key='place_id', low_memory=False)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [3]:
print('Number of tuples in A: {}'.format(len(A)))
print('Number of tuples in B: {}'.format(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): {}'.format(len(A) * len(B)))

Number of tuples in A: 6064
Number of tuples in B: 6130
Number of tuples in A X B (i.e the cartesian product): 37172320


In [6]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'place_id')

In [7]:
A1 = A[['id', 'name', 'location_display_address', 'phone', 
        'coordinates_latitude', 'coordinates_longitude']].copy()
B1 = B[['place_id', 'name', 'formatted_address', 'formatted_phone_number', 
        'geometry_location_lat', 'geometry_location_lng']].copy()

# Down Sampling

In [ ]:
# Down sampling the datasets to 1000 for development purposes
# Comment out for actual data
A1, B1 = em.down_sample(A[['id', 'name', 'location_display_address', 'phone', 
                           'coordinates_latitude', 'coordinates_longitude']], 
                        B[['place_id', 'name', 'formatted_address', 'formatted_phone_number', 
                           'geometry_location_lat', 'geometry_location_lng']], 
                        size=500, y_param=1, show_progress=True)

In [76]:
print('Number of tuples in A1: {}'.format(len(A1)))
print('Number of tuples in B1: {}'.format(len(B1)))
print('Number of tuples in A1 X B1 (i.e the cartesian product): {}'.format(len(A1) * len(B1)))

Number of tuples in A1: 6064
Number of tuples in B1: 6130
Number of tuples in A1 X B1 (i.e the cartesian product): 37172320


# Preprocess

In [8]:
def phone_string_process(x):
    try:
        return str(int(x)).lstrip('1')
    except ValueError:
        return x

In [9]:
A1['name'] = A1['name'].str.lower()
A1['name'] = A1['name'].str.replace('restaurant', '')

A1['location_display_address'] = A1['location_display_address'].str.lower()
A1['location_display_address'] = A1['location_display_address'].str.strip('[]')
A1['location_display_address'] = A1['location_display_address'].str.replace('\'', '')
A1['location_display_address'] = A1['location_display_address'].str.replace('#', '')
A1['location_display_address'] = A1['location_display_address'].str.replace(',', '')

A1['phone'] = A1['phone'].apply(phone_string_process)
                                                                            
A1 = A1.rename(columns={
    'location_display_address': 'address',
    'phone': 'phone', 
    'coordinates_latitude': 'lat',
    'coordinates_longitude': 'long', 
})

B1['name'] = B1['name'].str.lower()
B1['name'] = B1['name'].str.replace('restaurant', '')

B1['formatted_address'] = B1['formatted_address'].str.lower()
B1['formatted_address'] = B1['formatted_address'].str.replace('\'', '')
B1['formatted_address'] = B1['formatted_address'].str.replace('#', '')
B1['formatted_address'] = B1['formatted_address'].str.replace(',', '')

B1['formatted_phone_number'] = B1['formatted_phone_number'].str.replace('(', '')
B1['formatted_phone_number'] = B1['formatted_phone_number'].str.replace(')', '')
B1['formatted_phone_number'] = B1['formatted_phone_number'].str.replace('-', '')
B1['formatted_phone_number'] = B1['formatted_phone_number'].str.replace(' ', '')

B1 = B1.rename(columns={
    'formatted_address': 'address', 
    'formatted_phone_number': 'phone', 
    'geometry_location_lat': 'lat',
    'geometry_location_lng': 'long', 
})

In [10]:
yelp_sample = './../data/yelp_{}_sample.csv'.format(term)
google_sample = './../data/google_{}_sample.csv'.format(term)

In [11]:
# Saving the down sampled data for reuse.
em.to_csv_metadata(A1, yelp_sample)
em.to_csv_metadata(B1, google_sample)

File already exists at ./../data/yelp_restaurant_sample.csv; Overwriting it
File already exists at ./../data/google_restaurant_sample.csv; Overwriting it


True

In [12]:
# Data profiling and exploration
A1 = em.read_csv_metadata(yelp_sample, key='id', low_memory=False)
B1 = em.read_csv_metadata(google_sample, key='place_id', low_memory=False)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


# Block tables

In [13]:
A1['phone'] = A1['phone'].astype(str).str.replace('\.0', '')
B1['phone'] = B1['phone'].astype(str).str.replace('\.0', '')

In [14]:
def haversine(lon1, lat1, lon2, lat2, unit='km'):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    if unit == 'km':
        return 2 * 6371 * asin(sqrt(a))
    elif unit == 'mile':
        return (2 * 6371 * asin(sqrt(a))) / 1.609344 

In [15]:
jac = sm.Jaccard()
qg3_tok = sm.QgramTokenizer(qval=3)
def name_address_function(x, y):
    x_name = x['name']
    x_address = x['address']
    x_phone = x['phone']
    x_lat = x['lat']
    x_long = x['long']
    
    y_name = y['name']
    y_address = y['address']
    y_phone = y['phone']
    y_lat = y['lat']
    y_long = y['long']

    # Name must be available
    if type(x_name) != str or type(y_name) != str:
        return True
    
    distance = haversine(x_long, x_lat, y_long, y_lat)
    if distance > 5:
        return True
    else:
        if x_phone != 'nan' and y_phone != 'nan':
            if x_phone == y_phone:
                return False        
        
        t1 = qg3_tok.tokenize(x_name)
        t2 = qg3_tok.tokenize(y_name)
        name = bool(jac.get_raw_score(t1, t2) < 0.15)
        
        if distance > 1:
            if type(x_address) != str or type(y_address) != str:
                return True 
            else:
                t1 = qg3_tok.tokenize(x_address)
                t2 = qg3_tok.tokenize(y_address)
                address = bool(jac.get_raw_score(t1, t2) < 0.7)
                return address or name
        else:
            return name 

In [16]:
# Instantiate blackbox blocker
bb = em.BlackBoxBlocker()

# Set the black box function
bb.set_black_box_function(name_address_function)

C = bb.block_tables(A1, B1, 
                    l_output_attrs=['name', 'address', 'phone', 'lat', 'long'],
                    r_output_attrs=['name', 'address', 'phone', 'lat', 'long'], 
                    show_progress=True)

print('Pairs found: {}'.format(len(C)))

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:25:18


Pairs found: 11641


In [17]:
C.head(3)

,_id,ltable_id,rtable_place_id,ltable_name,ltable_address,ltable_phone,ltable_lat,ltable_long,rtable_name,rtable_address,rtable_phone,rtable_lat,rtable_long
0,0,AJ1SJkAVv1H6lA4pDnZ0dQ,ChIJvekefj5J2YAR57-HHVQJ3aI,ying li,342 w san ysidro blvd san ysidro ca 92173,6194282288,32.554284,-117.050823,ying li,342 w san ysidro blvd n san diego ca 92173 usa,6194282288,32.554340,-117.050951
1,1,uGt9jvS6tcPpYhJa1kDl1w,ChIJGVvXXaxJ2YARa8nlE2r3oU0,la cachonda cenaduria,347 w san ysidro blvd san diego ca 92173,nan,32.555684,-117.051453,la cachonda,347 w san ysidro blvd san diego ca 92173 usa,6192269077,32.555638,-117.051133
2,2,9BeEPZ4p0iEHYyKLYBLRBw,ChIJy6XT-j5J2YAR4kxAymYsYgM,zappy pizza,416 w san ysidro blvd san diego ca 92173,6196909232,32.554764,-117.052038,zappy pizza,416 w san ysidro blvd san ysidro ca 92173 usa,6196909232,32.554762,-117.052111


# Debug blocker output

In [22]:
# Display first few tuple pairs from the debug_blocker's output
# to make sure tuples are not blocking incorrectly
dbg = em.debug_blocker(C, A1, B1, output_size=50)
dbg

,_id,ltable_id,rtable_place_id,ltable_name,ltable_address,ltable_phone,rtable_name,rtable_address,rtable_phone
0,0,JyU1gtpMwTwvtPKXHe9_vg,ChIJl2fxNz4H3IARWidudy2KvW4,dr j's,9645 scranton rd san diego ca 92121,nan,park commons,9645 scranton rd san diego ca 92121 usa,nan
1,1,pmVf8dKK4ACHG4ogRalVAw,ChIJnY_6HeNR2YARN5aVz2wjFsY,sharky's,3030 plaza bonita rd national city ca 91950,nan,starbucks,3030 plaza bonita rd national city ca 91950 usa,6194752175
2,2,_7RajSyuMNOliMHF_YsN4w,ChIJv0giDDFV2YARkfyYqpGntP8,cafe terrace,7007 friars rd san diego ca 92108,nan,stacked,7007 friars rd 356 san diego ca 92108 usa,nan
3,3,LLT2FUyNMaxaKtzq8UdQvw,ChIJl2fxNz4H3IARWidudy2KvW4,fricken burgers,9645 scranton rd san diego ca 92121,nan,park commons,9645 scranton rd san diego ca 92121 usa,nan
4,4,VPyFEPULX3G9hLlQM5lRVg,ChIJj90A8fNV2YARWFIcQUhNzQQ,chick-fil-a,5323 mission center rd san diego ca 92108,6192911105,jer56w,5323 mission center rd san diego ca 92108 usa,nan
5,5,9CWFWngCc5-TFguk0a39Bg,ChIJKa3A8pr_24ARA8KqYL-7fqM,korean food,7655 clairemont mesa blvd san diego ca 92111,nan,kabobgee,clairemont mesa blvd san diego ca 92111 usa,nan
6,6,LZoTHYCaqSfrdnrPJLKGmw,ChIJickSuW1F2YARscGvqpTYzYk,wendy's,2260 otay lakes rd chula vista ca 91915,7603523102,pokebay,2260 otay lakes rd chula vista ca 91915 usa,6196518691
7,7,TgqlE7Y_OZY3OHq9FL-e2Q,ChIJl7qt8xUH3IARpIXzVKKrI5I,sorrento court,9450 scranton rd san diego ca 92121,nan,delicacy,9450 scranton rd 114a san diego ca 92121 usa,nan
8,8,dR0ZnLpRlnIIswZyGP-8Lg,ChIJ2xvq4w6q3oARsArmidIU0tc,draft,3105 ocean front walk san diego ca 92109,8582289305,cannonball,3105 ocean front walk san diego ca 92109 usa,8582289304
9,9,bMQsXDRNSgukAiWBphJDMA,ChIJPcE9Tnf524ARMVP_yXJzdrU,ihop,8440 mira mesa blvd san diego ca 92126,8582717995,jollibee,8440 mira mesa blvd san diego ca 92126 usa,nan


# Label the candidate set

In [23]:
# Sample candidate set
S = em.sample_table(C, 200)

In [24]:
# Label candidate S
G = em.label_table(S, 'label')

Column name (label) is not present in dataframe


In [25]:
sample_table_labeled = './../data/sample_block_labeled.csv'

In [26]:
# Saving the labeled data for future reuse
G.to_csv(sample_table_labeled, index=False)

In [27]:
# Load the labeled data
G = em.read_csv_metadata(sample_table_labeled, 
                         key='_id',
                         ltable=A1, rtable=B1, 
                         fk_ltable='ltable_id', fk_rtable='rtable_place_id')
print('Total labeled pairs: {}'.format(len(G)))

Metadata file is not present in the given path; proceeding to read the csv file.


Total labeled pairs: 200


In [28]:
G['ltable_phone'] = G['ltable_phone'].astype(str).str.replace('\.0', '')
G['rtable_phone'] = G['rtable_phone'].astype(str).str.replace('\.0', '')

In [29]:
G.head(3)

,_id,ltable_id,rtable_place_id,ltable_name,ltable_address,ltable_phone,ltable_lat,ltable_long,rtable_name,rtable_address,rtable_phone,rtable_lat,rtable_long,label
0,13,a6yxQHge4tcaKGV2lt4U3A,ChIJAcUnsB5J2YAROQmXCiUl9SU,tuetano taqueria,143 w san ysidro blvd san diego ca 92173,6198567013,32.553240,-117.045900,taqueria revolución,362 e san ysidro blvd san diego ca 92173 usa,6196903187,32.550714,-117.038357,0
1,84,VZXziy0PZlxWtq7czOXyIw,ChIJC2KlzwBH2YARwrf5itjOiCA,don lucio's mexican grill,2360 paseo de las americas ste 105 san diego ca 92154,6196611577,32.555074,-116.936443,don lucio's taco shop,2498 roll dr san diego ca 92154 usa,6196611577,32.555085,-116.936428,1
2,150,mBU-fwzP8-E_h58PheAuRA,ChIJ6RPgoAlH2YAR69Mu53CASyc,mariscos chuky's,av. jose lopez portillo pte. s/n 22436 tijuana baja california mexico,nan,32.538170,-116.930360,mariscos altamura,blvd. de las bellas artes lbnueva tijuana22435 tijuana b.c. blvd. de las bellas artes lb nueva t...,nan,32.543547,-116.935393,0


# Split the labeled data

In [30]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.8, random_state=1)
I = IJ['train']
J = IJ['test']

In [31]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [32]:
match_c = em.get_attr_corres(A1, B1)

In [33]:
match_tok = em.get_tokenizers_for_matching()
match_sim = em.get_sim_funs_for_matching()

atypes1 = em.get_attr_types(A1)
atypes2 = em.get_attr_types(B1)

feature_table = em.get_features(A1, B1, atypes1, atypes2, match_c, match_tok, match_sim)

In [34]:
feature_table = feature_table[(feature_table.left_attribute == 'name') | 
                              (feature_table.left_attribute == 'address') |
                              (feature_table.left_attribute == 'phone')]

In [35]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='label',
                            show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [36]:
# Evaluate ML matcher using CV=4
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H,
                           exclude_attrs=['_id', 'ltable_id', 'rtable_place_id', 'label'],
                           k=4,
                           target_attr='label', metric_to_select_matcher='f1', 
                           random_state=0)

result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.887363,0.874812,0.875082
1,RF,0.936688,0.910652,0.922408
2,SVM,0.968750,0.735464,0.835033
3,LinReg,0.907452,0.943985,0.921572
4,LogReg,0.958333,0.909461,0.932346
5,NaiveBayes,0.959375,0.926128,0.940948


#  Predict on test data

In [37]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, 
                            feature_table=feature_table,
                            attrs_after='label', 
                            show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [42]:
# Train using feature vectors from I 
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_place_id', 'label'], 
       target_attr='label')

In [43]:
# Predict on L 
predictions = lg.predict(table=L, 
                         exclude_attrs=['_id', 'ltable_id', 'rtable_place_id', 'label'], 
                         append=True, 
                         target_attr='predicted', 
                         inplace=False)

In [44]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 78.57% (11/14)
Recall : 91.67% (11/12)
F1 : 84.62%
False positives : 3 (out of 14 positive predictions)
False negatives : 1 (out of 26 negative predictions)


In [77]:
# Print the out to check the false positive/negative cases if any
final = J.copy() 
final['predicted'] = predictions['predicted']
final.head()

,_id,ltable_id,rtable_place_id,ltable_name,ltable_address,ltable_phone,ltable_lat,ltable_long,rtable_name,rtable_address,rtable_phone,rtable_lat,rtable_long,label,predicted
58,3810,qj3g9Wtn8rGChIRG5kcTKQ,ChIJQSo5cbpU2YARFW66pe7BXao,bankers hill bar & grill,3225 n harbor dr terminal 1 e fl 2 gate 7 san diego ca 92101,6192310222,32.732550,-117.196260,bankers hill bar +,2202 fourth ave san diego ca 92101 usa,6192310222,32.728410,-117.161357,0,0
40,2292,8PwGhzqDu_nvD7SNkU2jvw,ChIJOfZ1-QdT2YARkZS-h2MLGGY,hodads - petco park,100 park blvd san diego ca 92101,nan,32.706269,-117.156465,randy jones grill - petco park,100 park blvd san diego ca 92101 usa,nan,32.706373,-117.157145,0,0
34,1874,5n-7uAIGTWSVo5ZpYQWdWw,ChIJG_Rs7q5U2YARRdmnSHcpVEQ,the owl,602 broadway san diego ca 92101,6195010123,32.715950,-117.158950,the kebab shop,303 w beech st san diego ca 92101 usa,6195505481,32.720721,-117.165780,0,0
102,6772,dTK7MogmexEKaruicqSR4A,ChIJFz9l4AZY2YAR0YIosBlmiqc,hilberto's mexican food,9805 prospect ave santee ca 92071,6192588313,32.831040,-116.983310,santana's mexican food,10050 mission gorge rd santee ca 92071 usa,6196317680,32.838151,-116.977874,0,0
184,10959,teew2mRsrEirerG4VKNmVg,ChIJk9OXZz4H3IARGa8RC-3Oeb4,sher e punjab,9254 scranton rd ste 102 san diego ca 92121,8584582858,32.890894,-117.201661,sher-e-punjab -sorrento valley,9254 scranton rd 102 san diego ca 92121 usa,8584582858,32.891127,-117.201719,1,0


#  Predict on full blocking data

In [65]:
# Convert C into a set of feature vectors using feature table
L = em.extract_feature_vecs(C, 
                            feature_table=feature_table, 
                            show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


In [66]:
# Predict on L
predictions = lg.predict(table=L, 
                         exclude_attrs=['_id', 'ltable_id', 'rtable_place_id'], 
                         append=True, 
                         target_attr='predicted', 
                         inplace=False)

In [67]:
# Print the out to check the false positive/negative cases if any
data_all = C.copy() 
data_all['predicted'] = predictions['predicted']

In [69]:
data_positive = data_all[data_all['predicted'] == 1]

In [70]:
print('Total unique left table: {}'.format(data_positive['ltable_id'].nunique()))
print('Total unique right table: {}'.format(data_positive['rtable_place_id'].nunique()))
print('Total match: {}'.format(len(data_positive)))

Total unique left table: 4401
Total unique right table: 4430
Total match: 4501


In [71]:
# Unique value
unique_positive = data_positive.drop_duplicates(subset=['ltable_id'])
unique_positive = unique_positive.drop_duplicates(subset=['rtable_place_id'])

print('Total unique left table: {}'.format(unique_positive['ltable_id'].nunique()))
print('Total unique right table: {}'.format(unique_positive['rtable_place_id'].nunique()))
print('Total match: {}'.format(len(unique_positive)))

Total unique left table: 4349
Total unique right table: 4349
Total match: 4349


In [74]:
unique_positive.to_csv('./../data/{}_match.csv'.format(term), index=False)